## Python Interface for NYT API

I was originally interested in the Movies NYT API, but it seems like that is fully deprecated now. In it's place, they suggest using a filtered query to the Article Search API. Let's see what the endpoint gives us with the suggested filter query. 

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os



load_dotenv()
api_key = os.getenv('NYT_API_KEY')

url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
params = { 'fq': 'typeOfMaterials:Review AND section.name:Movies', 'api-key': api_key}

response = requests.get(url, params=params)
data = response.json()
print(data)



{'status': 'OK', 'copyright': 'Copyright (c) 2025 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'abstract': 'The film captures the friendship between an Iranian filmmaker and a Gaza City resident. They never actually meet but speak movingly via video calls.', 'byline': {'original': 'By Alissa Wilkinson'}, 'document_type': 'article', 'headline': {'main': '‘Put Your Soul on Your Hand and Walk’: An Extraordinary Voice (and Film)', 'kicker': 'Documentary Lens', 'print_headline': ''}, '_id': 'nyt://article/2e9d44f2-d0e0-5214-b908-16f29245bff9', 'keywords': [{'name': 'Subject', 'value': 'Israel-Gaza War (2023- )', 'rank': 1}, {'name': 'Subject', 'value': 'Documentary Films and Programs', 'rank': 2}, {'name': 'Title', 'value': 'Put Your Soul on Your Hand and Walk (Movie)', 'rank': 3}, {'name': 'Person', 'value': 'Farsi, Sepideh', 'rank': 4}, {'name': 'Subject', 'value': 'Refugees and Displaced Persons', 'rank': 5}, {'name': 'Location', 'value': 'Gaza Strip', 'rank'

We can see an interesting response structure. There’s a lot of information we won’t need, so let’s make sure to construct a DataFrame containing only the relevant data. First, we can use pandas’ `json_normalize()` function to flatten each item in the `docs` list into an easier-to-read DataFrame.

One tricky part is that the movie title is buried in the `keywords` list. It’s stored in the item where the `name` key is `'Title'`. We can use `apply` with a `lambda` function to locate the item in `keywords` where `name` is `'Title'` and extract the value from the `value` key.

Once we have the movie title extracted, we can construct a cleaner DataFrame, selecting and renaming only the columns we want, including the newly created `movie_title` column.


In [2]:
df = pd.json_normalize(data['response']['docs'])
df['movie_title'] = df['keywords'].apply(
    lambda lst: next((kw['value'] for kw in lst if kw['name'] == 'Title'), None)
)

df_movies_reviews = df[[
    'headline.main',
    'byline.original',
    'movie_title',
    'pub_date',
    'abstract',
    'word_count',
    'web_url'
]].rename(columns={
    'headline.main': 'headline',
    'byline.original': 'author',
    'pub_date': 'publication_date',
    'abstract': 'summary',
    'web_url': 'url'
})

df_movies_reviews.head(10)

,headline,author,movie_title,publication_date,summary,word_count,url
0,‘Put Your Soul on Your Hand and Walk’: An Extr...,By Alissa Wilkinson,Put Your Soul on Your Hand and Walk (Movie),2025-11-07T10:01:22Z,The film captures the friendship between an Ir...,608,https://www.nytimes.com/2025/11/07/movies/put-...
1,‘Sentimental Value’ Review: Joachim Trier’s Un...,By Manohla Dargis,Sentimental Value (Movie),2025-11-06T18:15:15Z,In this moving drama from the director of “The...,1233,https://www.nytimes.com/2025/11/06/movies/sent...
2,‘Sarah’s Oil’ Review: The Little Drilling Rig ...,By Natalia Winkelman,Sarah's Oil (Movie),2025-11-06T17:00:03Z,A young Black girl learns that her land allotm...,257,https://www.nytimes.com/2025/11/06/movies/sara...
3,‘Predator: Badlands’ Review: It Lives!,By Manohla Dargis,Predator: Badlands (Movie),2025-11-06T10:02:16Z,The latest installment from the director of “P...,787,https://www.nytimes.com/2025/11/06/movies/pred...
4,‘Peter Hujar’s Day’ Review: Tell Me Everything,By Jeannette Catsoulis,Peter Hujar's Day (Movie),2025-11-06T10:02:10Z,Ben Whishaw and Rebecca Hall invigorate this e...,498,https://www.nytimes.com/2025/11/06/movies/pete...
5,‘Nuremberg’ Review: Inside the Nazi Mind,By Ben Kenigsberg,Nuremberg (Movie),2025-11-06T10:02:10Z,This movie starring Rami Malek and Russell Cro...,561,https://www.nytimes.com/2025/11/06/movies/nure...
6,‘Modi’ Review: 3 Days in the Life of Modigliani,By Glenn Kenny,Modi: Three Days on the Wing of Madness (Movie),2025-11-06T10:02:06Z,"The movie, directed by Johnny Depp, adapted fr...",299,https://www.nytimes.com/2025/11/06/movies/modi...
7,‘Die My Love’ Review: Jennifer Lawrence in a M...,By Alissa Wilkinson,Die My Love (Movie),2025-11-06T10:02:04Z,The actress gives a career-defining performanc...,948,https://www.nytimes.com/2025/11/06/movies/die-...
8,‘Christy’ Review: The Lady in Pink? She Packs ...,By Lisa Kennedy,Christy (Movie),2025-11-06T10:02:02Z,"The ring is a treacherous place to be, but for...",553,https://www.nytimes.com/2025/11/06/movies/chri...
9,‘Caterpillar’ Review: Risking Their Sight to b...,By Ben Kenigsberg,Caterpillar (Movie),2025-11-06T10:01:59Z,The latest documentary from Liza Mandelup (“Ja...,287,https://www.nytimes.com/2025/11/06/movies/cate...
